# Estrategias para acelerar el código en R 
---

El autor nos informa sobre algunas desventajas del **bucle for** y sobre como podemos optimizar todo ese proceso. Solo basta ajustar algunos detalles para poder incrementar la velocidad.  

Ahora el autor nos da como ejemplo un conjunto de datos que lo almacenará en un data.frame.  
Inicializando el código de la siguiente manera:

In [2]:
# Create the data frame
col1 <- runif (12^5, 0, 2)
col2 <- rnorm (12^5, 0, 2)
col3 <- rpois (12^5, 3)
col4 <- rchisq (12^5, 2)
df <- data.frame (col1, col2, col3, col4)
dim(df) #dimensiones del data.frame df 

[1] 248832      4

Podemos observar que se tiene 248832 filas y 4 columnas, en total hay 995328 datos.  

Ahora el autor nos da el siguiente ejercicio. Sumar todas las filas del data.frame y verificar si la suma es mayor a 4 o menor que 4. Si fuese el primer caso, introducirá una quinta columna con el valor "mayor que 4" o " menor que 4".

In [ ]:
# Original R code: Before vectorization and pre-allocation
system.time({
  for (i in 1:nrow(df)) { # una iteración para cada fila 
    if ((df[i, "col1"] + df[i, "col2"] + df[i, "col3"] + df[i, "col4"]) > 4) { # check if > 4
      df[i, 5] <- "greater_than_4" # lo asigna a la quinta columna 
    } else {
      df[i, 5] <- "lesser_than_4" # lo asigna a la quinta columna
    }
  }

})

Timing stopped at: 977.1 23.32 1002


Tiempo de ejecución:

user | system | elapsed
-- | -- | --
377.264 | 0.516 | 378.366 

Ahora vamos a inicializar nuestra estructura de datos antes de usarla. Este método se llama vectorización. 

In [ ]:
# after vectorization and pre-allocation
output <- character (nrow(df)) # initialize output vector
system.time({
  for (i in 1:nrow(df)) {
    if ((df[i, "col1"] + df[i, "col2"] + df[i, "col3"] + df[i, "col4"]) > 4) {
      output[i] <- "greater_than_4"
    } else {
      output[i] <- "lesser_than_4"
    }
  }
df$output})

Tiempo de ejecución

user | system | elapsed
-- | -- | --
12.936| 0.008 | 13.007 

Comparando el tiempo de ejecución antes de la vectorización con el ya vectorizado, se observa que disminuye considerablemente. Haciéndolo más rápido.  

Ahora veamos la velocidad usando la condición `if-else` afuera del bucle `for`.

In [ ]:
# after vectorization and pre-allocation, taking the condition checking outside the loop.
output <- character (nrow(df))
condition <- (df$col1 + df$col2 + df$col3 + df$col4) > 4  # condition check outside the loop
system.time({
  for (i in 1:nrow(df)) {
    if (condition[i]) {
      output[i] <- "greater_than_4"
    } else {
      output[i] <- "lesser_than_4"
    }
  }
  df$output <- output
})

Tiempo de ejecución

user | system | elapsed
-- | -- | --
0.284| 0.000 |  0.283 

Ahora ejecutamos el código solo para las sentencias verdaderas. Así si hay una cantidad mayor de casos verdaderos posiblemente la velocidad será mejor. El código es de la siguiente forma:

In [ ]:
output <- character(nrow(df))
condition <- (df$col1 + df$col2 + df$col3 + df$col4) > 4
system.time({
  for (i in (1:nrow(df))[condition]) {  # El for solo es para condiciones verdaderas 
    if (condition[i]) {
      output[i] <- "greater_than_4"
    } else {
      output[i] <- "lesser_than_4"
    }
  }
  df$output })

Tiempo de ejecución

user | system | elapsed
-- | -- | --
0.204| 0.000| 0.206

Notamos que el tiempo es menor comparado con el caso anterior.  

Luego hacemos otro tipo de hacer menor el tiempo de velocidad. Para esto usamos una condición `ifelse()` que siempre sea posible. 

In [ ]:
#Código donde solo se usa un ifelse()
system.time({
  output <- ifelse ((df$col1 + df$col2 + df$col3 + df$col4) > 4, "greater_than_4", "lesser_than_4")
  df$output <- output
});

Tiempo de ejecución

user | system | elapsed
-- | -- | --
0.132| 0.004| 0.138

Notamos que el tiempo comparado con los anteriores `if` es menor. Ya que no hay una asignación previa y se verifica la condición en todos los casos.  

Otra forma de poder hallar la misma operación pero con una mayor velocidad es con `which()`.

In [ ]:
system.time({
  want = which(rowSums(df) > 4)
  output = rep("less than 4", times = nrow(df))
  output[want] = "greater than 4"
}) 

Tiempo de ejecución

user | system | elapsed
-- | -- | --
0.020 | 0.000| 0.017 

Usamos ahora la  función `apply` en vez del bucle `for`

In [ ]:
# funcion apply
system.time({
  myfunc <- function(x) {
    if ((x['col1'] + x['col2'] + x['col3'] + x['col4']) > 4) {
      "greater_than_4"
    } else {
      "lesser_than_4"
    }
  }
  output <- apply(df[, c(1:4)], 1, FUN=myfunc)  # apply 'myfunc' on every row
  df$output <- output
})

Tiempo de ejecución

user | system | elapsed
-- | -- | --
1.204 | 0.000|  1.211   

Comparando estos tiempos con el método de vectorización, códigos anteriores, el tiempo es mucho menor.  

Usamos ahora la función `cmpfun()`. 

In [ ]:
library(compiler) 
myFuncCmp <- cmpfun(myfunc)
system.time({
  output <- apply(df[, c (1:4)], 1, FUN=myFuncCmp)
})

Tiempo de ejecución

user | system | elapsed
-- | -- | --
1.06 | 0.00|  1.06   

Aquí notamos que el tiempo es regularmente bajo, comparado con `apply`, ya que este método se usa para funciones más complejas.  

Ahora usaremos Rcpp, es una libreria donde podemos exportar código de **C/C++** para usarlo junto con **R**.  
El código ahora usa alrededor de un millón de datos. 

In [ ]:
library(Rcpp) # llamando a la libreria Rcpp
sourceCpp("MyFunc.cpp")  # exportamos un código en C++ llamado Myfunc.cpp
system.time (output <- myFunc(df))

In [ ]:
// Esta es la función en c++
// Source for MyFunc.cpp
#include 
using namespace Rcpp;
// [[Rcpp::export]]
CharacterVector myFunc(DataFrame x) {
  NumericVector col1 = as(x["col1"]);
  NumericVector col2 = as(x["col2"]);
  NumericVector col3 = as(x["col3"]);
  NumericVector col4 = as(x["col4"]);
  int n = col1.size();
  CharacterVector out(n);
  for (int i=0; i 4){
      out[i] = "greater_than_4";
    } else {
      out[i] = "lesser_than_4";
    }
  }
  return out;
}

Viendo los tiempos de ejecución del código anterior, comparado con el código `ifelse()`, se concluye que es mucho menor. 

Usamos ahora la ejecución del código en paralelo de la siguiente manera.

In [ ]:
# proceso paralelo 
library(foreach)
library(doSNOW)
cl <- makeCluster(4, type="SOCK") # para 4 máquinas 
registerDoSNOW (cl)
condition <- (df$col1 + df$col2 + df$col3 + df$col4) > 4
# paralelizacion con vectorización
system.time({
  output <- foreach(i = 1:nrow(df), .combine=c) %dopar% {
    if (condition[i]) {
      return("greater_than_4")
    } else {
      return("lesser_than_4")
    }
  }
})
df$output <- output

Finalmente usamos una estructura de datos que consuma menos memoria. 

In [ ]:
dt <- data.table(df)  # creamos el data.table
system.time({
  for (i in 1:nrow (dt)) {
    if ((dt[i, col1] + dt[i, col2] + dt[i, col3] + dt[i, col4]) > 4) {
      dt[i, col5:="greater_than_4"]  # le asignamos "mayor_que_4" a la 5ta columna
    } else {
      dt[i, col5:="lesser_than_4"]  # le asignamos "menor_que_4" a la 5ta columna 
    }
  }
})


Comparando los datos con el primer ejemplo, sin vectorización y con `data.frame`, los tiempos con una estructura de datos `data.table` es mucho menor, ya que agiliza las operaciones al reducir la sobrecarga de la memoria.  

### Fuentes:  
 * Strategies to Speedup R Code. Selva Prabhakaran. Link: <https://datascienceplus.com/strategies-to-speedup-r-code/> 